<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/SCOTUS/mean_classification_scotus_no_pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
from datasets import load_dataset

In [ ]:
dataset=load_dataset('lex_glue', 'scotus')

In [ ]:
from transformers import AutoTokenizer
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('XXXX')"
tokenizer = AutoTokenizer.from_pretrained('danielsaggau/longformer_simcse_scotus', use_auth_token=True,use_fast=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
model_1 = AutoModelForSequenceClassification.from_pretrained('danielsaggau/longformer_simcse_scotus',use_auth_token=True, num_labels=14)

In [5]:
import torch 
from torch import nn

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [8]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_pred):
    metric1 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    micro1 = metric1.compute(predictions=predictions, references=labels, average="micro")["f1"]
    macro1 = metric1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return { "f1-micro": micro1, "f1-macro": macro1}

In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8) # fp16

In [10]:
!git clone https://github.com/danielsaggau/IR_LDC.git

Cloning into 'IR_LDC'...
remote: Enumerating objects: 1469, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 1469 (delta 327), reused 398 (delta 281), pack-reused 1004
Receiving objects: 100% (1469/1469), 3.70 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (895/895), done.


In [11]:
%cd IR_LDC

/content/IR_LDC


In [ ]:
!pip install wandb
import wandb
wandb.login()

In [13]:
%env WANDB_PROJECT=IR_LDC

env: WANDB_PROJECT=IR_LDC


In [14]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="/scotus_mean_no_pool",
    learning_rate=3e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    push_to_hub=True,
    report_to="wandb",
    fp16=True,
    #run_name="mean_ep10",
    metric_for_best_model="f1-micro",
    greater_is_better=True,
    load_best_model_at_end = True
)

In [ ]:
wandb.init(project="IR_LDC",name="mean_linear")

In [ ]:
model_1

In [16]:
from transformers import Trainer, EarlyStoppingCallback
import torch
trainer = Trainer(
    model=model_1,
    compute_metrics=compute_metrics,
    args=training_args,
    eval_dataset=tokenized_data['test'],
    train_dataset=tokenized_data["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,    
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)])
trainer.train()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/danielsaggau/scotus_mean_no_pool into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: text. If text are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *

Epoch,Training Loss,Validation Loss,F1-micro,F1-macro
1,1.355400,0.993715,0.708571,0.452174
2,0.675100,1.072144,0.708571,0.481249
3,0.485400,1.119652,0.747857,0.621353
4,0.345500,1.390274,0.727857,0.592823
5,0.237500,1.490348,0.739286,0.631017
6,0.131700,1.712229,0.724286,0.619723
7,0.108800,1.842729,0.727857,0.624504
8,0.055100,1.889849,0.737143,0.637925
9,0.037000,1.899125,0.735000,0.635387
10,0.025700,1.935226,0.733571,0.637913


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on C

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing 

TrainOutput(global_step=8340, training_loss=0.3256846584576211, metrics={'train_runtime': 7992.1853, 'train_samples_per_second': 6.256, 'train_steps_per_second': 1.044, 'total_flos': 2.937334206553747e+16, 'train_loss': 0.3256846584576211, 'epoch': 10.0})

In [17]:
trainer.evaluate(eval_dataset=tokenized_data['validation'])

The following columns in the evaluation set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: text. If text are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1400
  Batch size = 6
Initializing global attention on CLS token...


Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Initializing global attention on C

{'eval_loss': 0.9824504852294922,
 'eval_f1-micro': 0.7535714285714286,
 'eval_f1-macro': 0.6678811082623166,
 'eval_runtime': 63.8068,
 'eval_samples_per_second': 21.941,
 'eval_steps_per_second': 3.667,
 'epoch': 10.0}